In [1]:
from boggle.io import download_txt_file, get_word_list, process_word_list, calculate_letter_probs_from_word_list
from boggle.path import Path
from boggle.bisect import HasPrefix, AdaptPrefix, Prefix
from boggle.boggle import BoggleGrid
import numpy as np
import random
import time
import re
import bisect

In [2]:
MIN_WORD_LENGTH = 3

## Data

In [3]:
download_txt_file(txt_url='https://github.com/OpenTaal/opentaal-wordlist/raw/master/wordlist.txt', 
                  target_path='../data/wordlist.txt')
word_list = np.array(get_word_list(txt_path='../data/wordlist.txt'))
word_list = process_word_list(word_list, min_word_length=MIN_WORD_LENGTH)

## Utils

In [4]:
letter_probs = calculate_letter_probs_from_word_list(word_list)

In [8]:
boggle_grid = BoggleGrid(letter_probs = letter_probs, grid_size = (4,4)).get_grid()

In [6]:
class Boggle:
    
    def __init__(self, boggle_grid: np.array, letter_freqs: dict):
        """
        if boggle_grid = None, create random
        """
        if boggle_grid:
            self.boggle_grid = boggle_grid
        else:
            self.boggle_grid = self.create_random_grid(letter_freqs,grid_size)
            
                
    def __str__(self):
        return str(f'Path in {self.gridsize} grid: {self.nodes.tolist()}')
    
    def __repr__(self):
        return str(f'Path in {self.gridsize} grid: {self.nodes.tolist()}')
            

            

## Code

In [33]:
boggle_grid = BoggleGrid(letter_probs = letter_probs, grid_size = (4,4)).get_grid()

In [34]:
word_list_search = AdaptPrefix(word_list)

In [35]:
paths = []
starting_coords = [np.array([[x,y]]) for x in range(boggle_grid.shape[0]) for y in range(boggle_grid.shape[0])]
for xy in starting_coords: 
    paths.append(Path(nodes = xy,gridsize=(4,4)))

In [ ]:
all_words = []
all_words_found = False
while not all_words_found:
    paths = [y for x in [path.get_new_paths() for path in paths] for y in x]
    idx_to_keep = []
    for idx, path in enumerate(paths):
        word = path.grid_to_text(boggle_grid)
        left_index = bisect.bisect_left(word_list_search, Prefix(word))
        right_index = bisect.bisect_right(word_list_search, Prefix(word))
        words_with_prefix = word_list[left_index:right_index]
        # check if a word.
        if len(words_with_prefix)>0:
            if word in words_with_prefix:
                nodes = path.nodes
                all_words.append({'word': word, 'path': nodes})
                if len(words_with_prefix)==1:
                    pass
                else:
                    idx_to_keep.append(idx)
            else:
                idx_to_keep.append(idx)
    
    paths = [paths[i] for i in idx_to_keep]
    
    if len(idx_to_keep) == 0:
        all_words_found=True

In [ ]:
print('--- BOGGLE ---\n')
print(boggle_grid)

print('\n\n--- ALL WORDS ---\n')
print([word['word'] for word in all_words])